# Mobility data anlaysis (Matt)

In [1]:
# imports and setup
from   ast import literal_eval
import numpy as np
import os
import pandas as pd
from   scipy.stats import permutation_test
import string

data_dir = os.path.join('..', 'data')
derived_dir = os.path.join(data_dir, 'derived')

## Load data

In [2]:
def string_to_list(x):
    lst = literal_eval(x.replace(', nan', "', ZZZZ'").replace('[nan, ', "'['").replace(', nan]', "']'"))
    return [i for i in lst if i != 'ZZZZ']

conlit = pd.read_csv(
    os.path.join(derived_dir, 'CONLIT_CharData_AP_MW_5.csv.gz'), 
    index_col='book_id',
    converters={
        'gpe_places': string_to_list,
        'nongpe_places': string_to_list,
        'all_places': string_to_list,
        'gpe_sequences': string_to_list
    },
)
early = pd.read_csv(
    os.path.join(derived_dir, 'EARLY_CharData_MW.csv.gz'), 
    index_col='book_id',
    converters={
        'gpe_places': string_to_list,
        'nongpe_places': string_to_list,
        'all_places': string_to_list,
        'gpe_sequences': string_to_list
    },
)

## Feature differences by facet 

### CONLIT

In [3]:
def significance_label(pvalue, levels=[0.05, 0.01, 0.001]):
    if pvalue < levels[2]:
        return '***'
    elif pvalue < levels[1]:
        return '**'
    elif pvalue < levels[0]:
        return '*'
    else: 
        return ''

def diff_means(x, y, axis):
    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

def compare_facets(df, facets, cols_to_drop, statistic=diff_means):
    '''Compare statistic in numeric cols of df via permutation. Display results, return nothing.'''
    cols = df.select_dtypes(include='number').drop(columns=cols_to_drop).columns
    for facet in facets:
        print(f"\n==============\nFacet: {facet}\n==============")
        df = df.dropna(subset=[facet])
        for level in df[facet].unique():
            data = (df.loc[df[facet]==level, cols], df.loc[~(df[facet]==level), cols])
            level_mean = np.mean(data[0], axis=0)
            other_mean = np.mean(data[1], axis=0)
            output = pd.DataFrame(level_mean, columns=[level]).join(pd.DataFrame(other_mean, columns=['others']))
            res = permutation_test(data, statistic)
            output['diff'] = res.statistic
            output['p'] = res.pvalue
            output['sig'] = output['p'].apply(significance_label)
            display(output)

In [4]:
facet_by = ['inf_gender', 'Category', 'Genre']
ignore_numeric_cols = ['char_id', 'char_rank', 'num_words']
compare_facets(conlit, facet_by, ignore_numeric_cols, diff_means)


Facet: inf_gender


,she/her,others,diff,p,sig
char_count,3752.279775,3844.495428,-92.215652,0.3898,
num_gpe_places,16.356180,21.814954,-5.458775,0.0002,***
num_nongpe_places,122.757303,123.285637,-0.528334,0.8466,
num_all_places,138.620225,144.524476,-5.904251,0.1042,
dist_miles,51501.447552,72756.670191,-21255.222639,0.0002,***
Tokens,119516.989888,131900.656267,-12383.666379,0.0002,***
num_gpe_places_norm,0.000141,0.000170,-0.000029,0.0002,***
num_nongpe_places_norm,0.001127,0.001048,0.000079,0.0022,**
num_gpe_places_norm_byCharacter,0.005224,0.009569,-0.004345,0.0002,***
num_nongpe_places_norm_byCharacter,0.036178,0.037657,-0.001479,0.0392,*


,he/him/his,others,diff,p,sig
char_count,4012.726801,3475.705128,537.021673,0.0002,***
num_gpe_places,22.025360,16.663708,5.361652,0.0002,***
num_nongpe_places,127.342939,115.879684,11.463255,0.0002,***
num_all_places,148.785591,132.051282,16.734309,0.0002,***
dist_miles,73533.779439,52771.036370,20762.743069,0.0002,***
Tokens,133472.337752,118342.144970,15130.192782,0.0002,***
num_gpe_places_norm,0.000169,0.000146,0.000023,0.0002,***
num_nongpe_places_norm,0.001073,0.001074,-0.000001,0.9460,
num_gpe_places_norm_byCharacter,0.008396,0.007762,0.000634,0.1744,
num_nongpe_places_norm_byCharacter,0.036880,0.037687,-0.000807,0.2520,


,they/them/their,others,diff,p,sig
char_count,1490.612903,3924.422857,-2433.809954,0.0002,***
num_gpe_places,18.870968,20.103238,-1.232270,0.5464,
num_nongpe_places,66.516129,125.788190,-59.272061,0.0002,***
num_all_places,84.903226,145.339048,-60.435822,0.0002,***
dist_miles,61883.407727,66063.769770,-4180.362043,0.7108,
Tokens,109909.790323,128740.810286,-18831.019963,0.0008,***
num_gpe_places_norm,0.000183,0.000160,0.000024,0.0724,
num_nongpe_places_norm,0.000695,0.001091,-0.000396,0.0002,***
num_gpe_places_norm_byCharacter,0.025981,0.007321,0.018660,0.0002,***
num_nongpe_places_norm_byCharacter,0.048523,0.036642,0.011881,0.0002,***



Facet: Category


...

,PW,others,diff,p,sig
char_count,3177.618677,3880.336276,-702.717599,0.0002,***
num_gpe_places,14.801556,20.588684,-5.787127,0.0002,***
num_nongpe_places,121.428016,123.288523,-1.860508,0.7044,
num_all_places,135.782101,143.317416,-7.535315,0.1832,
dist_miles,49045.545388,67610.847930,-18565.302541,0.0024,**
Tokens,122261.315175,128472.023676,-6210.708501,0.1438,
num_gpe_places_norm,0.000134,0.000163,-0.000029,0.0006,***
num_nongpe_places_norm,0.001108,0.001070,0.000038,0.3358,
num_gpe_places_norm_byCharacter,0.005194,0.008469,-0.003275,0.0002,***
num_nongpe_places_norm_byCharacter,0.042319,0.036648,0.005672,0.0002,***


,MIX,others,diff,p,sig
char_count,1869.167539,3959.904222,-2090.736683,0.0002,***
num_gpe_places,25.450262,19.644253,5.806008,0.0012,**
num_nongpe_places,84.507853,125.997263,-41.489410,0.0002,***
num_all_places,109.403141,145.092651,-35.689509,0.0002,***
dist_miles,98662.101411,63427.082422,35235.018989,0.0002,***
Tokens,126088.335079,128026.023847,-1937.688768,0.7194,
num_gpe_places_norm,0.000204,0.000157,0.000047,0.0002,***
num_nongpe_places_norm,0.000745,0.001098,-0.000352,0.0002,***
num_gpe_places_norm_byCharacter,0.018654,0.007379,0.011275,0.0002,***
num_nongpe_places_norm_byCharacter,0.046469,0.036484,0.009985,0.0002,***


,MEM,others,diff,p,sig
char_count,4260.253275,3774.146032,486.107243,0.0110,*
num_gpe_places,37.877729,18.427381,19.450348,0.0002,***
num_nongpe_places,170.908297,118.771429,52.136868,0.0002,***
num_all_places,207.737991,136.694841,71.043150,0.0002,***
dist_miles,147811.848938,58429.374920,89382.474018,0.0002,***
Tokens,105611.161572,129916.065476,-24304.903904,0.0002,***
num_gpe_places_norm,0.000355,0.000143,0.000212,0.0002,***
num_nongpe_places_norm,0.001666,0.001019,0.000646,0.0002,***
num_gpe_places_norm_byCharacter,0.009739,0.008019,0.001720,0.0532,
num_nongpe_places_norm_byCharacter,0.041595,0.036777,0.004819,0.0002,***


,MID,others,diff,p,sig
char_count,3103.355422,3860.351916,-756.996495,0.0004,***
num_gpe_places,7.024096,20.884630,-13.860534,0.0002,***
num_nongpe_places,94.024096,124.984127,-30.960031,0.0002,***
num_all_places,100.753012,145.303136,-44.550124,0.0002,***
dist_miles,15068.771786,69140.349241,-54071.577454,0.0002,***
Tokens,84156.487952,130702.076655,-46545.588703,0.0002,***
num_gpe_places_norm,0.000089,0.000165,-0.000076,0.0002,***
num_nongpe_places_norm,0.001243,0.001062,0.000181,0.0002,***
num_gpe_places_norm_byCharacter,0.002412,0.008532,-0.006120,0.0002,***
num_nongpe_places_norm_byCharacter,0.031853,0.037520,-0.005667,0.0002,***


,BIO,others,diff,p,sig
char_count,4291.549223,3778.629499,512.919724,0.0152,*
num_gpe_places,59.735751,17.050861,42.684891,0.0002,***
num_nongpe_places,175.652850,119.147496,56.505354,0.0002,***
num_all_places,233.875648,135.721831,98.153817,0.0002,***
dist_miles,221164.856853,54149.499543,167015.357310,0.0002,***
Tokens,196532.989637,122708.362285,73824.627352,0.0002,***
num_gpe_places_norm,0.000331,0.000148,0.000184,0.0002,***
num_nongpe_places_norm,0.001036,0.001076,-0.000040,0.3918,
num_gpe_places_norm_byCharacter,0.015934,0.007576,0.008359,0.0002,***
num_nongpe_places_norm_byCharacter,0.046214,0.036496,0.009718,0.0002,***


### EARLY

In [5]:
facet_by = ['inf_gender']
ignore_numeric_cols = ['char_id', 'char_rank']
compare_facets(early, facet_by, ignore_numeric_cols, diff_means)


Facet: inf_gender


,he/him/his,others,diff,p,sig
char_count,2839.052502,3119.199823,-280.147322,0.0002,***
num_gpe_places,11.521179,10.780459,0.740720,0.0008,***
num_nongpe_places,91.156408,97.249559,-6.093150,0.0002,***
num_all_places,102.363125,107.705709,-5.342584,0.0002,***
dist_miles,29818.492460,28557.874476,1260.617984,0.2018,
Tokens,114450.075394,118587.624485,-4137.549091,0.0028,**


,she/her,others,diff,p,sig
char_count,3151.009276,2826.725071,324.284204,0.0002,***
num_gpe_places,10.840215,11.488369,-0.648153,0.0046,**
num_nongpe_places,98.153202,90.792001,7.361201,0.0002,***
num_all_places,108.666966,101.966807,6.700159,0.0002,***
dist_miles,28731.134875,29730.119332,-998.984457,0.3084,
Tokens,119471.410533,114079.797851,5391.612682,0.0004,***


,they/them/their,others,diff,p,sig
char_count,1283.641509,2936.268421,-1652.626912,0.0002,***
num_gpe_places,7.584906,11.304229,-3.719324,0.0028,**
num_nongpe_places,45.471698,93.329887,-47.858189,0.0002,***
num_all_places,52.849057,104.315977,-51.466921,0.0002,***
dist_miles,19234.576170,29468.620999,-10234.044829,0.0778,
Tokens,68470.773585,116000.479041,-47529.705456,0.0002,***


,ze/zem/zir/hir,others,diff,p,sig
char_count,111.666667,2928.867540,-2817.200873,0.0002,***
num_gpe_places,0.666667,11.288775,-10.622108,0.0014,**
num_nongpe_places,5.333333,93.117306,-87.783973,0.0002,***
num_all_places,6.000000,104.088400,-98.088400,0.0002,***
dist_miles,257.392635,29426.079307,-29168.686672,0.0134,*
Tokens,19447.666667,115791.927502,-96344.260836,0.0004,***
